**RDF creation of tensile test data using TTO**

In this notebook, data originating from tensile tests performed to an S355 steel sheet is read in to create an exemplary RDF data file that uses the [Tensile Test Ontology (TTO) version 3.0](https://github.com/materialdigital/tensile-test-ontology) as semantic backbone. Hence, the tensile test data available in 'classic' test formats is transformed to RDF data by creating triples using the RDFlib library and some helper functions. Therefore, values tokenized from the CSV are allocated to concepts of the TTO developed in the frame of the PMD project. Since the TTO is strongly based on the PMD Core Ontology - PMDco (namespace: [https://w3id.org/pmd/co](https://w3id.org/pmd/co)) - the PMDco will also be parsed and semantic concepts this ontology is related to will also be used, accordingly. The tensile test data is taken from a dataset published openly available on [Zenodo](https://zenodo.org/record/6778336).
The RDF data is directly saved in a graph. Subsequently, the graph is serialized and saved to a TTL and an RDF file. Furthermore, SPARQL queries may be performed for consistency checks.
Hence, all in all, this script is used to create a knowledge graph using vocabulary of several ontologies and tensile test data originating from real tests performed on an S355 steel in accordance with the corresponding test standard [ISO 6892-1:2019-11](https://www.beuth.de/de/norm/iso-6892-1/316885984).

# Installing and Importing relevant Python packages

In [1]:
%%capture
%pip install --upgrade pip
# Installing relevant python package rdflib
%pip install rdflib

In [2]:
# Import of relevant package parts
from rdflib import Graph, Literal, URIRef, BNode, Namespace, DC
from rdflib.term import Identifier
from rdflib.collection import Collection
from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
import rdflib.plugins.sparql.update
import pandas as pd
import io
import urllib.parse

# Definition of helper methods

In [3]:
# Definition of helper methods / functions - do not necessarily have to be used.

# Helper method 'add' used to write triples to an RDF graph
def add(s,p,o):

    # in this case p is "ObjectProperty"
    if o.find('http://')==0 or o.find('https://')==0:
        g.add( (URIRef(s), URIRef(p), URIRef(o)) )

    # in this case p is "DatatypeProperty"
    else:
        # if we can parse o as Float, just set the datatype
        try:
            g.add( (URIRef(s), URIRef(p), Literal(float(o), datatype=XSD.float) ))
        except:
            g.add( (URIRef(s), URIRef(p), Literal(o) ))


# Global counter possibly usable for the creation of arbitrary instance names (numbers)
instanceCounter = 0
def nextInstanceNum():
    global instanceCounter
    instanceCounter = instanceCounter + 1
    return str(instanceCounter)

# Read in tensile test data

In [4]:
# Read in data that is supposed to be mapped to the concepts of the ontology (TTO).
# The data is also findable as a dataset on tensile tests that is published freely available on Zenodo: https://zenodo.org/record/6778336
# Reading in data from URL

link_data = "https://raw.githubusercontent.com/materialdigital/tensile-test-ontology/main/data/S355_data.csv"
data_S333_read_in = pd.read_csv(link_data, sep=';')

# Graph creation using RDFlib

In [7]:
# Creating the RDF graph 'g'
g = Graph()

In [12]:
# Definition of IRI prefixes (namespaces)
base = Namespace("https://w3id.org/pmd/tto/")
g.bind("base", base)
pmdco = Namespace("https://w3id.org/pmd/co/")
g.bind("pmdco", pmdco)
unit = Namespace("http://qudt.org/vocab/unit/")
g.bind("unit", unit)
prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)
raw_data = Namespace("https://github.com/materialdigital/tensile-test-ontology/tree/3dc72e40d492402e5ccd46167b8c03c62599c5dc/data/primary_data/")
g.bind("raw_data", raw_data)
csvw = Namespace("http://www.w3.org/ns/csvw#")
g.bind("csvw", csvw)
datacite = Namespace("http://purl.org/spar/datacite/")
g.bind("datacite", datacite)
obo = Namespace("http://purl.obolibrary.org/obo/")
g.bind("obo", obo)
dct = Namespace("http://purl.org/dc/elements/1.1/")
g.bind("dct", dct)
time = Namespace("http://www.w3.org/2006/time#")
g.bind("time", time)

# A-Box namespace (namespace of instances)
prefix = Namespace("https://w3id.org/pmd/demodata/tensiletest_S355/")
g.bind("prefix", prefix)

# Turtle style abbreviation for RDF.type
a = RDF.type

# Create ontology and import TTO as basis
onto = URIRef(prefix)
g.add((onto, a, OWL.Ontology))
g.add((onto, OWL.imports, URIRef(base)))

# Add creators:
markus = URIRef("https://orcid.org/0000-0002-7094-5371")
bernd = URIRef("https://orcid.org/0000-0002-3717-7104")
joerg = URIRef("https://orcid.org/0000-0001-7192-7143")
philipp = URIRef("https://orcid.org/0000-0003-4971-3645")

g.add((onto, DC.creator, markus))
g.add((markus, RDFS.label, Literal("Markus Schilling", datatype=XSD.string)))
g.add((onto, DC.creator, bernd))
g.add((bernd, RDFS.label, Literal("Bernd Bayerlein", datatype=XSD.string)))
g.add((onto, DC.creator, joerg))
g.add((joerg, RDFS.label, Literal("Jörg Waitelonis", datatype=XSD.string)))
g.add((onto, DC.creator, philipp))
g.add((philipp, RDFS.label, Literal("Philipp von Hartrott", datatype=XSD.string)))

g.add((onto, DC.title, Literal("Tensile Test Ontology (TTO) data mapping example", datatype=XSD.string)))
g.add((onto, OWL.versionInfo, Literal("3.0.0", datatype=XSD.string)))

# Load data from read in above.
data = data_S333_read_in

# Creation of QUDT unit instances as instances of iao:measurement unit label
# mesurement unit label has IRI: obo.IAO_0000003 (see below for more details)
g.add((unit.MilliM, a, obo.IAO_0000003))
g.add((unit.SEC, a, obo.IAO_0000003))
g.add((unit.KiloN, a, obo.IAO_0000003))
g.add((unit.PERCENT, a, obo.IAO_0000003))
g.add((unit.MilliM2, a, obo.IAO_0000003))
g.add((unit.MegaPa, a, obo.IAO_0000003))
g.add((unit.GigaPa, a, obo.IAO_0000003))
g.add((unit.DEG_C, a, obo.IAO_0000003))

# Creation of triples
# Iterate over the full data table
for idx, row in data.iterrows():
  # Definition of tokens in accordance with columns of original csv file
  process_id                          = row["process"]
  testpiece_id                        = row["testpiece"]
  a1                                  = row["d1 / a1"]
  a2                                  = row["d2 / a2"]
  a3                                  = row["d3 / a3"]
  b1                                  = row["b1"]
  b2                                  = row["b2"]
  b3                                  = row["b3"]
  s1                                  = row["s1"]
  s2                                  = row["s2"]
  s3                                  = row["s3"]
  S0                                  = row["S0"]
  Rp02                                = row["Rp0,2"]
  ReH                                 = row["ReH"]
  Fm                                  = row["Fm"]
  Rm                                  = row["Rm"]
  E                                   = row["E"]
  L0                                  = row["L0"]
  Lu                                  = row["Lu"]
  A                                   = row["A"]
  au                                  = row["au"]
  bu                                  = row["bu"]
  Su                                  = row["Su"]
  Z                                   = row["Z"]
  operator                            = row["Operator"]
  date                                = row["date"]
  materialDescription                 = row["Werkstoff"]
  specimenShape                       = row["Probenform"]
  note                                = row["Note"]
  environmentalTemperature            = row["Umgebungstemperatur"]
  machineName                         = row["Prüfmaschinen-Name"]
  machineManufacturer                 = row["Prüfmaschinenhersteller"]
  machineType                         = row["Prüfmaschinen-Typ"]
  machineSerialNumber                 = row["Prüfmaschinen-Seriennummer"]
  machineStandard                     = row["Prüfmaschinen-Standard"]
  extensometerName                    = row["Extensometer"]
  extensometerSerialNumber            = row["Extensometer-Seriennummer"]
  extensometerStandard                = row["Extensometer-Standard"]
  micrometerGaugeManufacturer         = row["Mikrometerschraube-Hersteller"]
  caliperManufacturer                 = row["Messschieber-Hersteller"]
  caliperSerialNumber                 = row["Messschieber-Seriennummer"]
  strainRate                          = row["Dehngeschwindigkeit"]
  transitionPoint                     = row["Umschaltpunkt"]
  extensometerGaugeLength             = row["Extensometer-Nenngerätemesslänger"]
  loadCellMaxForce                    = row["Kraftmessdose-Kraftbereich"]
  project                             = row["Project"]
  standard                            = row["Norm"]
  rollingDirection                    = row["Walzrichtung"]
  location                            = row["Ort"]
  institute                           = row["Institut"]
  address                             = row["Adresse"]
 
  # Creation of triples for all instances (considering all connections between them) based on a bibliographic approach (RDFlib). There is about one "block" per instance.
  experimentIRI = URIRef(prefix + process_id)
  g.add((experimentIRI, a, obo.IAO_0020000))
  g.add((experimentIRI, RDFS.label, Literal(process_id, datatype=XSD.string)))  

  processIRI = URIRef(experimentIRI + "_process")
  g.add((processIRI, a, obo.OBI_0000011)) # Planned process
  g.add((processIRI, a, base.TensileTest)) # Tensile Test
  g.add((experimentIRI, obo.IAO_0000219, processIRI)) # Experiment ID denotes Process (Tensile Test)

  # Creation Process Plan and Process Plan Specification
  processPlanIRI = URIRef(experimentIRI + "_process_plan")
  g.add((processPlanIRI, a, obo.OBI_0000260)) # obi:Plan
  g.add((processIRI, obo.BFO_0000055, processPlanIRI)) # (Tensile Test) Process bfo:realizes the process plan

  processPlanSpecificationIRI = URIRef(experimentIRI + "_process_plan_specification")
  g.add((processPlanSpecificationIRI, a, obo.OBI_0000104)) # obi:Plan Specification
  g.add((processPlanIRI, obo.RO_0000059, processPlanSpecificationIRI)) # The process plan ro:concretizes a process plan specification


  # Advanced process description
  # In this section, the process of tensile testing is disassembled into individual parts, allowing a more precise process and subprocess description.

  # No. 1
  preMeasuringIRI = URIRef(experimentIRI + "_process_preMeasuring")
  g.add((preMeasuringIRI, a, obo.OBI_0000070)) # Assay (subclass of Planned process)
  g.add((preMeasuringIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((processIRI, obo.RO_0002224, preMeasuringIRI)) # (entire) process starts with
  
  # No. 2
  mountingIRI = URIRef(experimentIRI + "_process_mounting")
  g.add((mountingIRI, a, obo.OBI_0000011)) # Planned process
  g.add((mountingIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((mountingIRI, obo.RO_0002090, preMeasuringIRI)) # Mounting process immediately precedes preMeasuring process

  # No. 3
  gripsAlignmentIRI = URIRef(experimentIRI + "_process_gripsAlignment")
  g.add((gripsAlignmentIRI, a, obo.OBI_0000011)) # Planned process
  g.add((gripsAlignmentIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((gripsAlignmentIRI, obo.RO_0002090, mountingIRI)) # Grips Alignment process immediately precedes Mounting process

  # No. 4
  tensileTestISOIRI = URIRef(experimentIRI + "_process_tensileTestISO")
  g.add((tensileTestISOIRI, a, obo.OBI_0000011)) # Planned process
  g.add((tensileTestISOIRI, a, base.TensileTest)) # ISO Tensile testing refers to tensile testing
  g.add((tensileTestISOIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((tensileTestISOIRI, obo.RO_0002090, gripsAlignmentIRI)) # ISO Tensile Test process immediately precedes Grips Alignment process

  # No. 5
  demountingIRI = URIRef(experimentIRI + "_process_demounting")
  g.add((demountingIRI, a, obo.OBI_0000011)) # Planned process
  g.add((demountingIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((demountingIRI, obo.RO_0002090, tensileTestISOIRI)) # Demounting process immediately precedes ISO Tensile Test process

  # No. 6
  postMeasuringIRI = URIRef(experimentIRI + "_process_postMeasuring")
  g.add((postMeasuringIRI, a, obo.OBI_0000070)) # Assay (subclass of Planned process)
  g.add((postMeasuringIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((postMeasuringIRI, obo.RO_0002090, demountingIRI)) # Post Measuring process immediately precedes Demounting process

  # No. 7
  analysisIRI = URIRef(experimentIRI + "_process_analysis")
  g.add((analysisIRI, a, pmdco.ComputingProcess)) # Process is also a Computing process which is a subclass of planned process in acc. with new version of PMDco 3.0 datatransformation module
  # g.add((analysisIRI, a, obo.OBI_0000011)) # Planned process
  g.add((analysisIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((analysisIRI, obo.RO_0002090, postMeasuringIRI)) # Analysis process immediately precedes Post Measuring process

  
  # Object / test piece description
  # Test piece as input prior to tensile test
  testpieceIRI = URIRef(prefix  + f"testpiece/{testpiece_id}")
  g.add((testpieceIRI, a, pmdco.TestPiece))
  testpieceID_IRI = URIRef(prefix + f"testpiece_name/{testpiece_id}")
  g.add((testpieceID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceID_IRI, dct.identifier, Literal(testpiece_id, datatype=XSD.string)))
  g.add((testpieceID_IRI, obo.IAO_0000219, testpieceIRI)) # testpiece ID iao:denotes testpiece

  # Test piece(s) as output after tensile test (typically, 2 fractured parts will be formed)
  # Test piece after test 1
  testpieceAfterTest1IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_1")
  g.add((testpieceAfterTest1IRI, a, pmdco.TestPiece))
  testpieceAfterTest1ID_IRI = URIRef(testpieceID_IRI + "_afterTest_1")
  g.add((testpieceAfterTest1ID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceAfterTest1ID_IRI, dct.identifier, Literal(testpiece_id + "_afterTest_1", datatype=XSD.string)))
  g.add((testpieceAfterTest1ID_IRI, obo.IAO_0000219, testpieceAfterTest1IRI)) # testpiece after test 1 ID iao:denotes testpiece after test 1

  # Test piece after test 2
  testpieceAfterTest2IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_2")
  g.add((testpieceAfterTest2IRI, a, pmdco.TestPiece))
  testpieceAfterTest2ID_IRI = URIRef(testpieceID_IRI + "_afterTest_2")
  g.add((testpieceAfterTest2ID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceAfterTest2ID_IRI, dct.identifier, Literal(testpiece_id + "_afterTest_2", datatype=XSD.string)))
  g.add((testpieceAfterTest2ID_IRI, obo.IAO_0000219, testpieceAfterTest2IRI)) # testpiece after test 2 ID iao:denotes testpiece after test 2

  # Process - Test Piece Relations
  g.add((processIRI, obo.OBI_0000293, testpieceIRI)) # Process has specified input testpiece
  g.add((processIRI, obo.OBI_0000299, testpieceAfterTest1IRI)) # Process has specified output testpiece after test 1
  g.add((processIRI, obo.OBI_0000299, testpieceAfterTest2IRI)) # Process has specified output testpiece after test 2
  g.add((testpieceAfterTest1IRI, obo.RO_0001000, testpieceIRI)) # Testpiece after test 1 derives from Testpiece
  g.add((testpieceAfterTest2IRI, obo.RO_0001000, testpieceIRI)) # Testpiece after test 2 derives from Testpiece


  # The next section contains some material / test piece information.

  materialDescriptionIRI = URIRef(experimentIRI + "_material_description")
  g.add((materialDescriptionIRI, a, pmdco.MaterialDesignation)) # is a pmdco:MaterialDesignation as subclass of iao:identifier --> information content entity
  g.add((materialDescriptionIRI, dct.identifier, Literal(materialDescription, datatype=XSD.string)))
  g.add((materialDescriptionIRI, obo.IAO_0000219, testpieceIRI)) # Material Description (identifier) iao:denotes Test Piece (Test pieces after test are derived from test piece, anyways)

  specimenShapeIRI = URIRef(experimentIRI + "_specimen_shape")
  g.add((specimenShapeIRI, a, pmdco.Shape3D))
  g.add((testpieceIRI, obo.RO_0000086, specimenShapeIRI)) # Test piece ro:has quality specimen Shape

  specimenShapeTextualEntityIRI = URIRef(experimentIRI + "_specimen_shape_description")
  g.add((specimenShapeTextualEntityIRI, a, obo.IAO_0000300)) # is an iao:textual entity
  g.add((specimenShapeTextualEntityIRI, obo.OBI_0002815, Literal(specimenShape, datatype=XSD.string))) # Specimen Shape Textual Entity obi:has representation (Datatype Property used for textual information)
  g.add((specimenShapeTextualEntityIRI, obo.IAO_0000136, specimenShapeIRI)) # Specimen Shape Textual Entity iao:is about Specimen Shape  

  rollingDirectionInformationIRI = URIRef(experimentIRI + "_rolling_direction_information")
  g.add((rollingDirectionInformationIRI, a, obo.IAO_0000300)) # is an iao:textual entity
  g.add((rollingDirectionInformationIRI, obo.OBI_0002815, Literal(rollingDirection, datatype=XSD.string))) # Rolling Direction Information obi:has representation (Datatype Property used for textual information)
  g.add((rollingDirectionInformationIRI, obo.IAO_0000136, testpieceIRI)) # Rolling Direction Information Textual Entity iao:is about Test Piece

  # The next section contains some primary data.

  # Raw data / dataset
  # Using CSVW ontology vocabulary for raw / primary data set description
  datasetIRI = URIRef(experimentIRI + "resource/" + testpiece_id + "_dataset")
  g.add((datasetIRI, a, csvw.Table)) # is a csvw:Table
  g.add((datasetIRI, a, obo.IAO_0000100)) # is an iao:data set
  g.add((datasetIRI, DC.title, Literal(f"process/{process_id}" + f"testpiece_id/{testpiece_id}", datatype=XSD.string)))
  g.add((datasetIRI, csvw.url, Literal(raw_data + f"{testpiece_id}" + ".csv", datatype=XSD.string)))
  g.add((datasetIRI, DC.format, Literal("text/csv", datatype=XSD.string)))
  g.add((processIRI, obo.OBI_0000299, datasetIRI)) # Entire Tensile Test Process obi:has specified output dataset
  g.add((tensileTestISOIRI, obo.OBI_0000299, datasetIRI)) # ISO Tensile Test Process (subprocess) obi:has specified output dataset

  schemaIRI = URIRef(experimentIRI + testpiece_id + "_schema")
  g.add((schemaIRI, a, csvw.Schema))
  g.add((datasetIRI, csvw.tableSchema, schemaIRI))

  BNode1 = BNode()
  BNode2 = BNode()
  BNode3 = BNode()
  BNode4 = BNode()

  column1 = BNode()
  g.add((column1, a, csvw.Column))
  g.add((column1, a, time.Duration)) # is a time:Duration as subclass of time:TemporalDuration (owl:Class)
  g.add((column1, obo.IAO_0000039, unit.SEC)) # Column iao:has measurement label 
  g.add((column1, csvw.name, Literal("Time", datatype=XSD.string)))
  g.add((column1, DC.title, Literal("Time", datatype=XSD.string)))
  g.add((column1, csvw.datatype, Literal(1, datatype=XSD.float))) # here, also xsd:duration may be used; although, a decimal is given in the data!
  g.add((column1, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column2 = BNode()
  g.add((column2, a, csvw.Column))
  g.add((column2, a, base.CrossheadSeparation))
  g.add((column2, obo.IAO_0000039, unit.MilliM)) # Column iao:has measurement label 
  g.add((column2, csvw.name, Literal("Crosshead travel", datatype=XSD.string)))
  g.add((column2, DC.title, Literal("Crosshead Separation", datatype=XSD.string)))
  g.add((column2, csvw.datatype, Literal(2, datatype=XSD.float)))
  g.add((column2, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column3 = BNode()
  g.add((column3, a, csvw.Column))
  g.add((column3, a, pmdco.Force))
  g.add((column3, obo.IAO_0000039, unit.KiloN)) # Column iao:has measurement label 
  g.add((column3, csvw.name, Literal("Force", datatype=XSD.string)))
  g.add((column3, DC.title, Literal("Force", datatype=XSD.string)))
  g.add((column3, csvw.datatype, Literal(3, datatype=XSD.float)))
  g.add((column3, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column4 = BNode()
  g.add((column4, a, csvw.Column))
  g.add((column4, a, base.PercentageExtension))
  g.add((column4, obo.IAO_0000039, unit.PERCENT)) # Column iao:has measurement label 
  g.add((column4, csvw.name, Literal("Strain", datatype=XSD.string)))
  g.add((column4, DC.title, Literal("Percentage Extension", datatype=XSD.string)))
  g.add((column4, csvw.datatype, Literal(4, datatype=XSD.float)))
  g.add((column4, csvw.headerCount, Literal(2, datatype=XSD.float)))

  g.add((schemaIRI, csvw.column, BNode1))
  g.add((BNode1, RDF.first, column1))
  g.add((BNode1, RDF.rest, BNode2))
  g.add((BNode2, RDF.first, column2))
  g.add((BNode2, RDF.rest, BNode3))
  g.add((BNode3, RDF.first, column3))
  g.add((BNode3, RDF.rest, BNode4))
  g.add((BNode4, RDF.first, column4))
  g.add((BNode4, RDF.rest, RDF.nil))

  # Rest of primary data.
  # a1
  a1IRI_value_specification = URIRef(experimentIRI + "_thickness_a1_value_specification")
  g.add((a1IRI_value_specification, a, obo.OBI_0001931)) # a1 measurement value specification is a obi:scalar value specification
  g.add((a1IRI_value_specification, obo.OBI_0001937, Literal(a1, datatype=XSD.float))) # a1IRI_value_specification has specified numeric value a1
  g.add((a1IRI_value_specification, obo.IAO_0000039, unit.MilliM)) # a1 measurement value specification iao:has measurement unit label unit:MilliM (QUDT)
  g.add((a1IRI_value_specification, obo.IAO_0000136, testpieceIRI)) # a1 measurement value specification iao:is about Testpiece

  a1IRI = URIRef(experimentIRI + "_thickness_a1_scalar_measurement_datum")
  g.add((a1IRI, a, obo.IAO_0000032)) # a1 is a iao:scalar measurement datum
  g.add((a1IRI, obo.OBI_0001938, a1IRI_value_specification)) # a1 scalar measurement datum obi:has specified numeric value a1 measurement value specification
  g.add((processIRI, obo.OBI_0000299, a1IRI)) # Process has specified output a1
  g.add((preMeasuringIRI, obo.OBI_0000299, a1IRI)) # preMeasuring process has specified output a1
  g.add((a1IRI, obo.IAO_0000136, testpieceIRI)) # a1 scalar measurement datum iao:is about testpiece

  a1IRI_quality = URIRef(experimentIRI + "_thickness_a1")
  g.add((a1IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a1IRI_quality, obo.IAO_0000417, a1IRI)) # a1 quality iao:is quality measured as a1
  g.add((testpieceIRI, obo.RO_0000086, a1IRI_quality)) # Testpiece ro:has quality a1 quality

  # a2
  a2IRI_value_specification = URIRef(experimentIRI + "_thickness_a2_value_specification")
  g.add((a2IRI_value_specification, a, obo.OBI_0001931))
  g.add((a2IRI_value_specification, obo.OBI_0001937, Literal(a2, datatype=XSD.float)))
  g.add((a2IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((a2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  a2IRI = URIRef(experimentIRI + "_thickness_a2_scalar_measurement_datum")
  g.add((a2IRI, a, obo.IAO_0000032))
  g.add((a2IRI, obo.OBI_0001938, a2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, a2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, a2IRI)) # preMeasuring process has specified output a2
  g.add((a2IRI, obo.IAO_0000136, testpieceIRI))

  a2IRI_quality = URIRef(experimentIRI + "_thickness_a2")
  g.add((a2IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a2IRI_quality, obo.IAO_0000417, a2IRI))
  g.add((testpieceIRI, obo.RO_0000086, a2IRI_quality))

  # a3
  a3IRI_value_specification = URIRef(experimentIRI + "_thickness_a3_value_specification")
  g.add((a3IRI_value_specification, a, obo.OBI_0001931))
  g.add((a3IRI_value_specification, obo.OBI_0001937, Literal(a3, datatype=XSD.float)))
  g.add((a3IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((a3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  a3IRI = URIRef(experimentIRI + "_thickness_a3_scalar_measurement_datum")
  g.add((a3IRI, a, obo.IAO_0000032))
  g.add((a3IRI, obo.OBI_0001938, a3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, a3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, a3IRI)) # preMeasuring process has specified output a3
  g.add((a3IRI, obo.IAO_0000136, testpieceIRI))

  a3IRI_quality = URIRef(experimentIRI + "_thickness_a3")
  g.add((a3IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a3IRI_quality, obo.IAO_0000417, a3IRI))
  g.add((testpieceIRI, obo.RO_0000086, a3IRI_quality))

  # b1
  b1IRI_value_specification = URIRef(experimentIRI + "_width_b1_value_specification")
  g.add((b1IRI_value_specification, a, obo.OBI_0001931))
  g.add((b1IRI_value_specification, obo.OBI_0001937, Literal(b1, datatype=XSD.float)))
  g.add((b1IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b1IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b1IRI = URIRef(experimentIRI + "_width_b1_scalar_measurement_datum")
  g.add((b1IRI, a, obo.IAO_0000032))
  g.add((b1IRI, obo.OBI_0001938, b1IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b1IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b1IRI)) # preMeasuring process has specified output b1
  g.add((b1IRI, obo.IAO_0000136, testpieceIRI))

  b1IRI_quality = URIRef(experimentIRI + "_width_b1")
  g.add((b1IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b1IRI_quality, obo.IAO_0000417, b1IRI))
  g.add((testpieceIRI, obo.RO_0000086, b1IRI_quality))

  # b2
  b2IRI_value_specification = URIRef(experimentIRI + "_width_b2_value_specification")
  g.add((b2IRI_value_specification, a, obo.OBI_0001931))
  g.add((b2IRI_value_specification, obo.OBI_0001937, Literal(b2, datatype=XSD.float)))
  g.add((b2IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b2IRI = URIRef(experimentIRI + "_width_b2_scalar_measurement_datum")
  g.add((b2IRI, a, obo.IAO_0000032))
  g.add((b2IRI, obo.OBI_0001938, b2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b2IRI)) # preMeasuring process has specified output b2
  g.add((b2IRI, obo.IAO_0000136, testpieceIRI))

  b2IRI_quality = URIRef(experimentIRI + "_width_b2")
  g.add((b2IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b2IRI_quality, obo.IAO_0000417, b2IRI))
  g.add((testpieceIRI, obo.RO_0000086, b2IRI_quality))

  # b3
  b3IRI_value_specification = URIRef(experimentIRI + "_width_b3_value_specification")
  g.add((b3IRI_value_specification, a, obo.OBI_0001931))
  g.add((b3IRI_value_specification, obo.OBI_0001937, Literal(b3, datatype=XSD.float)))
  g.add((b3IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b3IRI = URIRef(experimentIRI + "_width_b3_scalar_measurement_datum")
  g.add((b3IRI, a, obo.IAO_0000032))
  g.add((b3IRI, obo.OBI_0001938, b3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b3IRI)) # preMeasuring process has specified output b3
  g.add((b3IRI, obo.IAO_0000136, testpieceIRI))

  b3IRI_quality = URIRef(experimentIRI + "_width_b3")
  g.add((b3IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b3IRI_quality, obo.IAO_0000417, b3IRI))
  g.add((testpieceIRI, obo.RO_0000086, b3IRI_quality))

  # s1
  s1IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s1_value_specification")
  g.add((s1IRI_value_specification, a, obo.OBI_0001931))
  g.add((s1IRI_value_specification, obo.OBI_0001937, Literal(s1, datatype=XSD.float)))
  g.add((s1IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s1IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s1IRI = URIRef(experimentIRI + "_crossSectionArea_s1_scalar_measurement_datum")
  g.add((s1IRI, a, obo.IAO_0000032))
  g.add((s1IRI, obo.OBI_0001938, s1IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s1IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s1IRI)) # preMeasuring process has specified output s1
  g.add((s1IRI, obo.IAO_0000136, testpieceIRI))

  s1IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s1")
  g.add((s1IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s1IRI_quality, obo.IAO_0000417, s1IRI))
  g.add((testpieceIRI, obo.RO_0000086, s1IRI_quality))

  # s2
  s2IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s2_value_specification")
  g.add((s2IRI_value_specification, a, obo.OBI_0001931))
  g.add((s2IRI_value_specification, obo.OBI_0001937, Literal(s2, datatype=XSD.float)))
  g.add((s2IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s2IRI = URIRef(experimentIRI + "_crossSectionArea_s2_scalar_measurement_datum")
  g.add((s2IRI, a, obo.IAO_0000032))
  g.add((s2IRI, obo.OBI_0001938, s2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s2IRI)) # preMeasuring process has specified output s2
  g.add((s2IRI, obo.IAO_0000136, testpieceIRI))

  s2IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s2")
  g.add((s2IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s2IRI_quality, obo.IAO_0000417, s2IRI))
  g.add((testpieceIRI, obo.RO_0000086, s2IRI_quality))

  # s3
  s3IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s3_value_specification")
  g.add((s3IRI_value_specification, a, obo.OBI_0001931))
  g.add((s3IRI_value_specification, obo.OBI_0001937, Literal(s3, datatype=XSD.float)))
  g.add((s3IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s3IRI = URIRef(experimentIRI + "_crossSectionArea_s3_scalar_measurement_datum")
  g.add((s3IRI, a, obo.IAO_0000032))
  g.add((s3IRI, obo.OBI_0001938, s3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s3IRI)) # preMeasuring process has specified output s3
  g.add((s3IRI, obo.IAO_0000136, testpieceIRI))

  s3IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s3")
  g.add((s3IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s3IRI_quality, obo.IAO_0000417, s3IRI))
  g.add((testpieceIRI, obo.RO_0000086, s3IRI_quality))

  # S0
  s0IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s0_value_specification")
  g.add((s0IRI_value_specification, a, obo.OBI_0001931))
  g.add((s0IRI_value_specification, obo.OBI_0001937, Literal(S0, datatype=XSD.float)))
  g.add((s0IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s0IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s0IRI = URIRef(experimentIRI + "_crossSectionArea_s0_scalar_measurement_datum")
  g.add((s0IRI, a, obo.IAO_0000032))
  g.add((s0IRI, obo.OBI_0001938, s0IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s0IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s0IRI)) # preMeasuring process has specified output S0
  g.add((s0IRI, obo.IAO_0000136, testpieceIRI))

  s0IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s0")
  g.add((s0IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s0IRI_quality, obo.IAO_0000417, s0IRI))
  g.add((testpieceIRI, obo.RO_0000086, s0IRI_quality))

  # L0
  L0IRI_value_specification = URIRef(experimentIRI + "_originalGaugeLength_L0_value_specification")
  g.add((L0IRI_value_specification, a, obo.OBI_0001931))
  g.add((L0IRI_value_specification, obo.OBI_0001937, Literal(L0, datatype=XSD.float)))
  g.add((L0IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((L0IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  L0IRI = URIRef(experimentIRI + "_originalGaugeLength_L0_scalar_measurement_datum")
  g.add((L0IRI, a, obo.IAO_0000032))
  g.add((L0IRI, obo.OBI_0001938, L0IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, L0IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, L0IRI)) # preMeasuring process has specified output L0
  g.add((L0IRI, obo.IAO_0000136, testpieceIRI))

  L0IRI_quality = URIRef(experimentIRI + "_originalGaugeLength_L0")
  g.add((L0IRI_quality, a, base.OriginalGaugeLength)) # Original Gauge Length used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((L0IRI_quality, obo.IAO_0000417, L0IRI))
  g.add((testpieceIRI, obo.RO_0000086, L0IRI_quality))

  # Lu
  LuIRI_value_specification = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu_value_specification")
  g.add((LuIRI_value_specification, a, obo.OBI_0001931))
  g.add((LuIRI_value_specification, obo.OBI_0001937, Literal(Lu, datatype=XSD.float)))
  g.add((LuIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((LuIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  LuIRI = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu_scalar_measurement_datum")
  g.add((LuIRI, a, obo.IAO_0000032))
  g.add((LuIRI, obo.OBI_0001938, LuIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, LuIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, LuIRI)) # postMeasuring process has specified output Lu
  g.add((LuIRI, obo.IAO_0000136, testpieceIRI))

  LuIRI_quality = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu")
  g.add((LuIRI_quality, a, base.FinalGaugeLengthAfterFracture)) # Final Gauge Length After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((LuIRI_quality, obo.IAO_0000417, LuIRI))
  g.add((testpieceIRI, obo.RO_0000086, LuIRI_quality))

  # au
  auIRI_value_specification = URIRef(experimentIRI + "_thicknessAfterFracture_au_value_specification")
  g.add((auIRI_value_specification, a, obo.OBI_0001931))
  g.add((auIRI_value_specification, obo.OBI_0001937, Literal(au, datatype=XSD.float)))
  g.add((auIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((auIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  auIRI = URIRef(experimentIRI + "_thicknessAfterFracture_au_scalar_measurement_datum")
  g.add((auIRI, a, obo.IAO_0000032))
  g.add((auIRI, obo.OBI_0001938, auIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, auIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, auIRI)) # postMeasuring process has specified output au
  g.add((auIRI, obo.IAO_0000136, testpieceIRI))

  auIRI_quality = URIRef(experimentIRI + "_thicknessAfterFracture_au")
  g.add((auIRI_quality, a, base.ThicknessAfterFracture)) # Thickness After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((auIRI_quality, obo.IAO_0000417, auIRI))
  g.add((testpieceIRI, obo.RO_0000086, auIRI_quality))

  # bu
  buIRI_value_specification = URIRef(experimentIRI + "_widthAfterFracture_bu_value_specification")
  g.add((buIRI_value_specification, a, obo.OBI_0001931))
  g.add((buIRI_value_specification, obo.OBI_0001937, Literal(bu, datatype=XSD.float)))
  g.add((buIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((buIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  buIRI = URIRef(experimentIRI + "_widthAfterFracture_bu_scalar_measurement_datum")
  g.add((buIRI, a, obo.IAO_0000032))
  g.add((buIRI, obo.OBI_0001938, buIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, buIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, buIRI)) # postMeasuring process has specified output bu
  g.add((buIRI, obo.IAO_0000136, testpieceIRI))

  buIRI_quality = URIRef(experimentIRI + "_widthAfterFracture_bu")
  g.add((buIRI_quality, a, base.WidthAfterFracture)) # Width After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((buIRI_quality, obo.IAO_0000417, buIRI))
  g.add((testpieceIRI, obo.RO_0000086, buIRI_quality))

  # Su
  SuIRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_Su_value_specification")
  g.add((SuIRI_value_specification, a, obo.OBI_0001931))
  g.add((SuIRI_value_specification, obo.OBI_0001937, Literal(Su, datatype=XSD.float)))
  g.add((SuIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((SuIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  SuIRI = URIRef(experimentIRI + "_crossSectionArea_Su_scalar_measurement_datum")
  g.add((SuIRI, a, obo.IAO_0000032))
  g.add((SuIRI, obo.OBI_0001938, SuIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, SuIRI))
  g.add((analysisIRI, obo.OBI_0000299, SuIRI)) # Analysis process has specified output Su, since Su is calculated within this process
  g.add((SuIRI, obo.IAO_0000136, testpieceIRI))

  SuIRI_quality = URIRef(experimentIRI + "_crossSectionArea_Su")
  g.add((SuIRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((SuIRI_quality, obo.IAO_0000417, SuIRI))
  g.add((testpieceIRI, obo.RO_0000086, SuIRI_quality))

  # The next section contains some secondary data.

  # Rp02
  Rp02IRI_value_specification = URIRef(experimentIRI + "_proofStrength_Rp02_value_specification")
  g.add((Rp02IRI_value_specification, a, obo.OBI_0001931))
  g.add((Rp02IRI_value_specification, obo.OBI_0001937, Literal(Rp02, datatype=XSD.float)))
  g.add((Rp02IRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((Rp02IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  Rp02IRI = URIRef(experimentIRI + "_proofStrength_Rp02_scalar_measurement_datum")
  g.add((Rp02IRI, a, obo.IAO_0000032))
  g.add((Rp02IRI, obo.OBI_0001938, Rp02IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, Rp02IRI))
  g.add((analysisIRI, obo.OBI_0000299, Rp02IRI)) # Analysis process has specified output Rp02, since it is calculated / computed within this process
  g.add((Rp02IRI, obo.IAO_0000136, testpieceIRI))

  Rp02IRI_quality = URIRef(experimentIRI + "_proofStrength_Rp02")
  g.add((Rp02IRI_quality, a, base.Rp02)) # Rp02 used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((Rp02IRI_quality, obo.IAO_0000417, Rp02IRI))
  g.add((testpieceIRI, obo.RO_0000086, Rp02IRI_quality))

  # ReH
  ReHIRI_value_specification = URIRef(experimentIRI + "_upperYieldStrength_ReH_value_specification")
  g.add((ReHIRI_value_specification, a, obo.OBI_0001931))
  g.add((ReHIRI_value_specification, obo.OBI_0001937, Literal(ReH, datatype=XSD.float)))
  g.add((ReHIRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((ReHIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  ReHIRI = URIRef(experimentIRI + "_upperYieldStrength_ReH_scalar_measurement_datum")
  g.add((ReHIRI, a, obo.IAO_0000032))
  g.add((ReHIRI, obo.OBI_0001938, ReHIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, ReHIRI))
  g.add((analysisIRI, obo.OBI_0000299, ReHIRI)) # Analysis process has specified output ReH, since it is calculated / computed within this process
  g.add((ReHIRI, obo.IAO_0000136, testpieceIRI))

  ReHIRI_quality = URIRef(experimentIRI + "_upperYieldStrength_ReH")
  g.add((ReHIRI_quality, a, base.UpperYieldStrength)) # Upper Yield Strength used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((ReHIRI_quality, obo.IAO_0000417, ReHIRI))
  g.add((testpieceIRI, obo.RO_0000086, ReHIRI_quality))

  # Fm
  FmIRI_value_specification = URIRef(experimentIRI + "_maximumForce_Fm_value_specification")
  g.add((FmIRI_value_specification, a, obo.OBI_0001931))
  g.add((FmIRI_value_specification, obo.OBI_0001937, Literal(Fm, datatype=XSD.float)))
  g.add((FmIRI_value_specification, obo.IAO_0000039, unit.kiloN))
  g.add((FmIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  FmIRI = URIRef(experimentIRI + "_maximumForce_Fm_scalar_measurement_datum")
  g.add((FmIRI, a, obo.IAO_0000032))
  g.add((FmIRI, obo.OBI_0001938, FmIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, FmIRI))
  g.add((analysisIRI, obo.OBI_0000299, FmIRI)) # Analysis process has specified output Fm, since it is calculated / computed within this process
  g.add((FmIRI, obo.IAO_0000136, testpieceIRI))

  FmIRI_quality = URIRef(experimentIRI + "_maximumForce_Fm")
  g.add((FmIRI_quality, a, base.MaximumForce)) # Maximum Force used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((FmIRI_quality, obo.IAO_0000417, FmIRI))
  g.add((testpieceIRI, obo.RO_0000086, FmIRI_quality))

  # Rm
  RmIRI_value_specification = URIRef(experimentIRI + "_tensileStrength_Rm_value_specification")
  g.add((RmIRI_value_specification, a, obo.OBI_0001931))
  g.add((RmIRI_value_specification, obo.OBI_0001937, Literal(Rm, datatype=XSD.float)))
  g.add((RmIRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((RmIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  RmIRI = URIRef(experimentIRI + "_tensileStrength_Rm_scalar_measurement_datum")
  g.add((RmIRI, a, obo.IAO_0000032))
  g.add((RmIRI, obo.OBI_0001938, RmIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, RmIRI))
  g.add((analysisIRI, obo.OBI_0000299, RmIRI)) # Analysis process has specified output Rm, since it is calculated / computed within this process
  g.add((RmIRI, obo.IAO_0000136, testpieceIRI))

  RmIRI_quality = URIRef(experimentIRI + "_tensileStrength_Rm")
  g.add((RmIRI_quality, a, base.TensileStrength)) # Tensile Strength used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((RmIRI_quality, obo.IAO_0000417, RmIRI))
  g.add((testpieceIRI, obo.RO_0000086, RmIRI_quality))

  # E
  EIRI_value_specification = URIRef(experimentIRI + "_modulusOfElasticity_E_value_specification")
  g.add((EIRI_value_specification, a, obo.OBI_0001931))
  g.add((EIRI_value_specification, obo.OBI_0001937, Literal(E, datatype=XSD.float)))
  g.add((EIRI_value_specification, obo.IAO_0000039, unit.GigaPa))
  g.add((EIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  EIRI = URIRef(experimentIRI + "_modulusOfElasticity_E_scalar_measurement_datum")
  g.add((EIRI, a, obo.IAO_0000032))
  g.add((EIRI, obo.OBI_0001938, EIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, EIRI))
  g.add((analysisIRI, obo.OBI_0000299, EIRI)) # Analysis process has specified output E, since it is calculated / computed within this process
  g.add((EIRI, obo.IAO_0000136, testpieceIRI))

  EIRI_quality = URIRef(experimentIRI + "_modulusOfElasticity_E")
  g.add((EIRI_quality, a, pmdco.ModulusOfElasticity)) # Modulus Of Elasticity (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((EIRI_quality, obo.IAO_0000417, EIRI))
  g.add((testpieceIRI, obo.RO_0000086, EIRI_quality))

  # Slope of the elastic part
  slopeElasticPartIRI_value_specification = URIRef(experimentIRI + "_slopeOfTheElasticPart_me_value_specification")
  g.add((slopeElasticPartIRI_value_specification, a, obo.OBI_0001931))
  g.add((slopeElasticPartIRI_value_specification, obo.OBI_0001937, Literal(E, datatype=XSD.float)))
  g.add((slopeElasticPartIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  slopeElasticPartIRI = URIRef(experimentIRI + "_slopeOfTheElasticPart_me_scalar_measurement_datum")
  g.add((slopeElasticPartIRI, a, obo.IAO_0000032))
  g.add((slopeElasticPartIRI, obo.OBI_0001938, slopeElasticPartIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, slopeElasticPartIRI))
  g.add((analysisIRI, obo.OBI_0000299, slopeElasticPartIRI)) # Analysis process has specified output slope of the elastic part, since it is calculated / computed within this process
  g.add((slopeElasticPartIRI, obo.IAO_0000136, testpieceIRI))

  slopeElasticPartIRI_quality = URIRef(experimentIRI + "_slopeOfTheElasticPart_me")
  g.add((slopeElasticPartIRI_quality, a, base.SlopeOfTheElasticPart)) # Slope Of The Elastic Part used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((slopeElasticPartIRI_quality, obo.IAO_0000417, slopeElasticPartIRI))
  g.add((testpieceIRI, obo.RO_0000086, slopeElasticPartIRI_quality))

  # A
  AIRI_value_specification = URIRef(experimentIRI + "_percentageElongationAfterFracture_A_value_specification")
  g.add((AIRI_value_specification, a, obo.OBI_0001931))
  g.add((AIRI_value_specification, obo.OBI_0001937, Literal(A, datatype=XSD.float)))
  g.add((AIRI_value_specification, obo.IAO_0000039, unit.PERCENT))
  g.add((AIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  AIRI = URIRef(experimentIRI + "_percentageElongationAfterFracture_A_scalar_measurement_datum")
  g.add((AIRI, a, obo.IAO_0000032))
  g.add((AIRI, obo.OBI_0001938, AIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, AIRI))
  g.add((analysisIRI, obo.OBI_0000299, AIRI)) # Analysis process has specified output A, since it is calculated / computed within this process
  g.add((AIRI, obo.IAO_0000136, testpieceIRI))

  AIRI_quality = URIRef(experimentIRI + "_percentageElongationAfterFracture_A")
  g.add((AIRI_quality, a, base.PercentageElongationAfterFracture)) # Percentage Elongation After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((AIRI_quality, obo.IAO_0000417, AIRI))
  g.add((testpieceIRI, obo.RO_0000086, AIRI_quality))

  # Z
  ZIRI_value_specification = URIRef(experimentIRI + "_percentageReductionOfArea_Z_value_specification")
  g.add((ZIRI_value_specification, a, obo.OBI_0001931))
  g.add((ZIRI_value_specification, obo.OBI_0001937, Literal(Z, datatype=XSD.float)))
  g.add((ZIRI_value_specification, obo.IAO_0000039, unit.PERCENT))
  g.add((ZIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  ZIRI = URIRef(experimentIRI + "_percentageReductionOfArea_Z_scalar_measurement_datum")
  g.add((ZIRI, a, obo.IAO_0000032))
  g.add((ZIRI, obo.OBI_0001938, ZIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, ZIRI))
  g.add((analysisIRI, obo.OBI_0000299, ZIRI)) # Analysis process has specified output Z, since it is calculated / computed within this process
  g.add((ZIRI, obo.IAO_0000136, testpieceIRI))

  ZIRI_quality = URIRef(experimentIRI + "_percentageReductionOfArea_Z")
  g.add((ZIRI_quality, a, base.PercentageReductionOfArea)) # Percentage Reduction Of Area used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((ZIRI_quality, obo.IAO_0000417, ZIRI))
  g.add((testpieceIRI, obo.RO_0000086, ZIRI_quality))

  # The next section contains metadata concerning the process itself.

  # Environmental Temperature
  temperatureIRI_value_specification = URIRef(experimentIRI + "_environmentalTemperature_value_specification")
  g.add((temperatureIRI_value_specification, a, obo.OBI_0001931))
  g.add((temperatureIRI_value_specification, obo.OBI_0001937, Literal(environmentalTemperature, datatype=XSD.float)))
  g.add((temperatureIRI_value_specification, obo.IAO_0000039, unit.DEG_C))

  temperatureIRI = URIRef(experimentIRI + "_environmentalTemperature_scalar_measurement_datum")
  g.add((temperatureIRI, a, obo.IAO_0000032)) # is scalar measurement datum
  g.add((temperatureIRI, obo.OBI_0001938, temperatureIRI_value_specification)) # temperatureIRI obi:has value specification temperatureIRI_value_specification
  g.add((processIRI, obo.RO_0000057, temperatureIRI)) # Tensile Test Process ro:has participant Environmental Temperature scalar measurement datum

  temperatureIRI_quality = URIRef(experimentIRI + "_environmentalTemperature")
  g.add((temperatureIRI_quality, a, pmdco.EnvironmentalTemperature)) # Environmental Temperature (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((temperatureIRI_quality, obo.IAO_0000417, temperatureIRI)) # temperatureIRI_quality iao:is quality measured as temperatureIRI
  g.add((temperatureIRI_value_specification, obo.OBI_0001927, temperatureIRI_quality)) # temperatureIRI_value_specification obi:specifies value of temperatureIRI_quality
  
  dateIRI = URIRef(experimentIRI + "_date")
  g.add((dateIRI, a, time.Instant)) # is a time:instant (Definition: A temporal entity with zero extent or duration)
  g.add((dateIRI, time.inXSDDate, Literal(date, datatype=XSD.date))) # Date is given as xsd:date
  g.add((processIRI, time.hasTime, dateIRI)) # Process has Time date

  # Project and Funding information
  projectIRI = URIRef(experimentIRI + "_project")
  g.add((projectIRI, a, pmdco.Project)) # is a pmdco:Project as subclass of iao:planned process
  g.add((projectIRI, obo.BFO_0000051, processIRI)) # Project bfo:has part Tensile Test Process

  projectIdentifierIRI = URIRef(experimentIRI + "_project_ID")
  g.add((projectIdentifierIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((projectIdentifierIRI, dct.identifier, Literal(project, datatype=XSD.string)))
  g.add((projectIdentifierIRI, obo.IAO_0000219, projectIRI)) # Project identifier iao:denotes Project

  FunderRoleIRI = URIRef(experimentIRI + "funder_role")
  g.add((FunderRoleIRI, a, pmdco.FunderRole)) # is a pmdco:FunderRole
  g.add((FunderRoleIRI, obo.BFO_0000054, projectIRI)) # Funder Role is bfo:realized in Project

  BMBF_IRI = URIRef(experimentIRI + "_funder")
  g.add((BMBF_IRI, a, pmdco.Organization)) # is a pmdco:Organization
  g.add((BMBF_IRI, obo.RO_0000087, FunderRoleIRI)) # Funder (BMBF) ro:has role Funder Role
  g.add((projectIRI, obo.RO_0002608, BMBF_IRI)) # Project ro:process has causal agent Funder (BMBF)

  BMBFNameIRI = URIRef(experimentIRI + "_funder_name")
  g.add((BMBFNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((BMBFNameIRI, dct.identifier, Literal("Bundesministerium für Bildung und Forschung (BMBF)@de", datatype=XSD.string)))
  g.add((BMBFNameIRI, dct.identifier, Literal("Federal Ministry of Education and Research@en", datatype=XSD.string)))
  g.add((BMBFNameIRI, obo.IAO_0000219, BMBF_IRI)) # BMBF Name (identifier) iao:denotes Funder (BMBF)

  BMBFIdentifierNumberIRI = URIRef(experimentIRI + "_funder_identifier_number")
  g.add((BMBFIdentifierNumberIRI, a, obo.IAO_0000578)) # is an iao:centrally registered identifier
  g.add((BMBFIdentifierNumberIRI, dct.identifier, Literal("Q492234", datatype=XSD.string)))
  g.add((BMBFIdentifierNumberIRI, obo.IAO_0000219, BMBF_IRI)) # BMBF Identifier Number iao:denotes Funder (BMBF)

  fundingProjectIRI = URIRef(experimentIRI + "_funding_project")
  g.add((fundingProjectIRI, a, pmdco.Project)) # is a pmdco:Project as subclass of iao:planned process
  g.add((fundingProjectIRI, obo.BFO_0000051, projectIRI)) # Funding Project bfo:has part Project

  FundingProjectIdentifierIRI = URIRef(experimentIRI + "_funding_project_ID")
  g.add((FundingProjectIdentifierIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((FundingProjectIdentifierIRI, dct.identifier, Literal("13XP5094", datatype=XSD.string)))
  g.add((FundingProjectIdentifierIRI, obo.IAO_0000219, fundingProjectIRI)) # Funding Project identifier iao:denotes Funding Project

  # Operator
  operatorRoleIRI = URIRef(experimentIRI + "_operator_role")
  g.add((operatorRoleIRI, a, pmdco.OperatorRole)) # is a pmdco:OperatorRole
  g.add((operatorRoleIRI, obo.BFO_0000054, processIRI)) # Operator Role is bfo:realized in (tensile testing) process
    
  operatorIRI = URIRef(experimentIRI + "_operator")
  g.add((operatorIRI, a, pmdco.Operator)) # is a pmdco:Operator
  g.add((operatorIRI, obo.RO_0000087, operatorRoleIRI)) # Operator ro:has role Operator Role
  g.add((processIRI, obo.RO_0002608, operatorIRI)) # Tensile Test Process ro:prcoess has causal agent Operator
  g.add((operatorIRI, obo.RO_0002215, processIRI)) # Operator is ro:capable of Tensile Test Process

  operatorNameIRI = URIRef(experimentIRI + "_operator_name")
  g.add((operatorNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((operatorNameIRI, dct.identifier, Literal(operator, datatype=XSD.string)))
  g.add((operatorNameIRI, obo.IAO_0000219, operatorIRI)) # Operator Name (identifier) iao:denotes Operator

  # Test Standard
  standardInformationIRI = URIRef(experimentIRI + "_test_standard")
  g.add((standardInformationIRI, a, obo.IAO_0000300)) # is an iao:textual entity
  g.add((standardInformationIRI, obo.OBI_0002815, Literal(standard, datatype=XSD.string))) # Test Standard Information obi:has representation (Datatype Property used for textual information)
  g.add((standardInformationIRI, obo.IAO_0000136, processIRI)) # Test Standard Information Textual Entity iao:is about Tensile Test Process

  # Note
  noteIRI = URIRef(experimentIRI + "_note")
  g.add((noteIRI, a, obo.IAO_0000300)) # is an iao:textual entity
  g.add((noteIRI, obo.OBI_0002815, Literal(note, datatype=XSD.string))) # Note obi:has representation (Datatype Property used for textual information)
  g.add((noteIRI, obo.IAO_0000136, processIRI)) # Note Textual Entity iao:is about Tensile Test Process

  # The next section contains metadata on location, address, institute, etc.
  # 'Static' data such as institution and location get a static URI (the same URI / instance for every process regarded)
  # (there is only one of the explicitely specified institution and location that is referred to, respectively)
  institutionRoleIRI = URIRef(experimentIRI + "_institute_role")
  g.add((institutionRoleIRI, a, pmdco.DepartmentRole)) # is a pmdco:DepartmentRole
  g.add((institutionRoleIRI, obo.BFO_0000054, processIRI)) # Institution Role is bfo:realized in (tensile testing) process
    
  institutionIRI = URIRef(experimentIRI + "_institute")
  g.add((institutionIRI, a, pmdco.Organization)) # is a pmdco:Organization
  g.add((institutionIRI, obo.RO_0000087, institutionRoleIRI)) # Institution ro:has role Institution Role
  g.add((processIRI, obo.RO_0002608, institutionIRI)) # Tensile Test Process ro:prcoess has causal agent Institution

  institutionNameIRI = URIRef(experimentIRI + "_institute_BAM")
  g.add((institutionNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((institutionNameIRI, dct.identifier, Literal(institute, datatype=XSD.string)))
  g.add((institutionNameIRI, obo.IAO_0000219, institutionIRI)) # Institution Name (identifier) iao:denotes Institution

  addressIRI = URIRef(experimentIRI + "_institute_BAM_address")
  g.add((addressIRI, a, obo.IAO_0000422)) # is an iao:postal address as subclass of textual entity
  g.add((addressIRI, obo.OBI_0002815, Literal(f"{address}" + " " + f"{location}", datatype=XSD.string))) # Address obi:has representation 
  g.add((addressIRI, obo.IAO_0000136, institutionIRI)) # Address iao:is about institution

  # The next section contains metadata concerning machines, components (processing nodes) used.
  # 'Static' data such as manufacturers and serial numbers get a static URI (the same URI / instance for every process regarded)
  # Caution: If several machines were used, the machineIRI will have to be changed:
  # Then, it will have to contain the machine designation in the IRI (cf. e.g. institute)

  # Tensile Testing Machine
  machineFunctionIRI = URIRef(experimentIRI + "_tensile_testing_machine_function")
  g.add((machineFunctionIRI, a, base.TensileTestingFunction)) # is a tto:tensile testing function
  g.add((machineFunctionIRI, obo.BFO_0000054, processIRI)) # tensile testing function is bfo:realized in (tensile testing) process
  g.add((machineFunctionIRI, obo.BFO_0000054, tensileTestISOIRI)) # tensile testing function is bfo:realized in ISO tensile testing process (subprocess)

  machineIRI = URIRef(experimentIRI + "_tensile_testing_machine")
  g.add((machineIRI, a, base.TensileTestingMachine)) # Machine is a tto:Tensile Testing Machine
  g.add((machineIRI, obo.RO_0000085, machineFunctionIRI)) # Tensile Testing Machine ro:has function Tensile Testing Function
  g.add((machineIRI, obo.RO_0002215, processIRI)) # Tensile Testing Machine is ro:capable of (tensile testing) process (can realize the process)
  g.add((machineIRI, obo.RO_0002215, tensileTestISOIRI)) # Tensile Testing Machine is ro:capable of ISO tensile testing process (subprocess, can realize the process)
  g.add((processIRI, obo.RO_0002608, machineIRI)) # Process (tensile testing) ro:process has causal agent Tensile Testing Machine
  g.add((tensileTestISOIRI, obo.RO_0002608, machineIRI)) # ISO tensile testing process ro:process has causal agent Tensile Testing Machine  

  machineNameIRI = URIRef(experimentIRI + "_tensile_testing_machine_name")
  g.add((machineNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((machineNameIRI, dct.identifier, Literal(machineName, datatype=XSD.string)))
  g.add((machineNameIRI, obo.IAO_0000219, machineIRI)) # Machine Name (identifier) iao:denotes Machine (Tensile Testing Machine)

  machineManufacturerIRI = URIRef(experimentIRI + "_machine_manufacturer")
  g.add((machineManufacturerIRI, a, pmdco.Manufacturer)) # is a Manufacturer which is a subclass of Organization --> material entity
  g.add((machineManufacturerIRI, dct.identifier, Literal(machineManufacturer, datatype=XSD.string)))
  g.add((machineIRI, obo.OBI_0000304, machineManufacturerIRI)) # Tensile Testing Machine obi:is_manufactured_by Manufacturer

  machineManufacturerRoleIRI = URIRef(experimentIRI + "_machine_manufacturer_role")
  g.add((machineManufacturerRoleIRI, a, obo.OBI_0000571)) # is a Manufacturer Role which is a subclass of role --> realizable entity (in OBI)
  g.add((machineManufacturerIRI, obo.RO_0000087, machineManufacturerRoleIRI)) # The Manufacturer ro:has role Manufacturer Role

  machineTypeIRI = URIRef(experimentIRI + "_tensile_testing_machine_type")
  g.add((machineTypeIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((machineTypeIRI, dct.identifier, Literal(machineType, datatype=XSD.string)))
  g.add((machineTypeIRI, obo.IAO_0000219, machineIRI)) # Machine Type (identifier) iao:denotes Machine (Tensile Testing Machine)

  machineSerialNumberIRI = URIRef(experimentIRI + "_tensile_testing_machine_serial_number")
  g.add((machineSerialNumberIRI, a, obo.IAO_0000131)) # is an iao:serial number as subclass of iao:symbol --> iao:information content entity
  g.add((machineSerialNumberIRI, dct.identifier, Literal(machineSerialNumber, datatype=XSD.string)))
  g.add((machineSerialNumberIRI, obo.IAO_0000219, machineIRI)) # Machine Serial Number (symbol) iao:denotes Machine (Tensile Testing Machine)

  machineStandardIRI = URIRef(experimentIRI + "_tensile_testing_machine_standard")
  g.add((machineStandardIRI, a, pmdco.DeviceSpecification)) # is a pmdco:DeviceSpecification as subclass of iao:directive information entity --> iao:information content entity
  g.add((machineStandardIRI, dct.identifier, Literal(machineStandard, datatype=XSD.string)))
  g.add((machineStandardIRI, obo.IAO_0000136, machineIRI)) # Machine Standard (symbol) iao:is about Machine (Tensile Testing Machine)

  # Strain Rate
  strainRateIRI_value_specification = URIRef(experimentIRI + "_strain_rate_value_specification")
  g.add((strainRateIRI_value_specification, a, obo.OBI_0001931)) # Strain rate measurement value specification is a obi:scalar value specification
  g.add((strainRateIRI_value_specification, obo.OBI_0001937, Literal(strainRate, datatype=XSD.float))) # strainRateIRI_value_specification has specified numeric value strainRate

  strainRate_device_setting_IRI = URIRef(experimentIRI + "_strain_rate_device_setting")
  g.add((strainRate_device_setting_IRI, a, obo.OBI_0000654)) # is a obi:device setting
  g.add((strainRateIRI_value_specification, obo.OBI_0001927, strainRate_device_setting_IRI)) # strainRateIRI_value_specification obi:specifies value of strainRate device setting
  g.add((machineIRI, obo.RO_0000086, strainRate_device_setting_IRI)) # Tensile Testing Machine ro:has quality strainRate device setting

  strainRateQualityIRI = URIRef(experimentIRI + "_strain_rate_quality")
  g.add((strainRateQualityIRI, a, base.StrainRate)) # is a tto:StrainRate as subclass of tto:TestingRate --> quality
  g.add((strainRate_device_setting_IRI, obo.RO_0002566, strainRateQualityIRI)) # strainRate_device_setting ro:causally influences Strain Rate Quality
  g.add((strainRateIRI_value_specification, obo.OBI_0001927, strainRateQualityIRI)) # strainRateIRI_value_specification obi:specifies value of Strain Rate Quality
  g.add((machineIRI, obo.RO_0000086, strainRateQualityIRI)) # Tensile Testing Machine ro:has quality Strain Rate Quality

  # Transition Point
  transitionPointIRI_value_specification = URIRef(experimentIRI + "_transition_point_value_specification")
  g.add((transitionPointIRI_value_specification, a, obo.OBI_0001931)) # Transition point measurement value specification is a obi:scalar value specification
  g.add((transitionPointIRI_value_specification, obo.OBI_0001937, Literal(transitionPoint, datatype=XSD.float))) # transitionPointIRI_value_specification has specified numeric value transitionPoint
  g.add((transitionPointIRI_value_specification, obo.IAO_0000039, unit.PERCENT)) # Transition point measurement value specification iao:has measurement unit label unit:PERCENT (QUDT)

  transitionPoint_device_setting_IRI = URIRef(experimentIRI + "_transition_point_device_setting")
  g.add((transitionPoint_device_setting_IRI, a, obo.OBI_0000654)) # is a obi:device setting
  g.add((transitionPointIRI_value_specification, obo.OBI_0001927, transitionPoint_device_setting_IRI)) # transitionPointIRI_value_specification obi:specifies value of transitionPoint device setting
  g.add((machineIRI, obo.RO_0000086, transitionPoint_device_setting_IRI)) # Tensile Testing Machine ro:has quality transitionPoint device setting

  transitionPointQualityIRI = URIRef(experimentIRI + "_strain_rate_quality")
  g.add((transitionPointQualityIRI, a, base.TransitionPoint)) # is a tto:TransitionPoint --> quality
  g.add((transitionPoint_device_setting_IRI, obo.RO_0002566, transitionPointQualityIRI)) # transitionPoint_device_setting ro:causally influences Transition Point Quality
  g.add((transitionPointIRI_value_specification, obo.OBI_0001927, transitionPointQualityIRI)) # transitionPointIRI_value_specification obi:specifies value of Transition Point Quality
  g.add((machineIRI, obo.RO_0000086, transitionPointQualityIRI)) # Tensile Testing Machine ro:has quality Transition Point Quality

  # Extensometer
  extensometerFunctionIRI = URIRef(experimentIRI + "_extensometer_function")
  g.add((extensometerFunctionIRI, a, obo.OBI_0000453)) # is a obi:measure function
  g.add((extensometerFunctionIRI, obo.BFO_0000054, processIRI)) # measure function (of extensometer) is bfo:realized in (tensile testing) process
  g.add((extensometerFunctionIRI, obo.BFO_0000054, tensileTestISOIRI)) # extensometer function is bfo:realized in ISO tensile testing process (subprocess)

  extensometerIRI = URIRef(experimentIRI + "_extensometer")
  g.add((extensometerIRI, a, pmdco.Extensometer)) # Extensometer is a pmdco:Extensometer
  g.add((extensometerIRI, obo.RO_0000085, extensometerFunctionIRI)) # Extensometer ro:has function extensometer function
  g.add((machineIRI, obo.RO_0002180, extensometerIRI)) # Tensile Testing Machine ro:has component Extensometer

  extensometerNameIRI = URIRef(experimentIRI + "_extensometer_name")
  g.add((extensometerNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((extensometerNameIRI, dct.identifier, Literal(extensometerName, datatype=XSD.string)))
  g.add((extensometerNameIRI, obo.IAO_0000219, extensometerIRI)) # Extensometer Name (identifier) iao:denotes Extensometer

  extensometerSerialNumberIRI = URIRef(experimentIRI + "_extensometer_serial_number")
  g.add((extensometerSerialNumberIRI, a, obo.IAO_0000131)) # is an iao:serial number as subclass of iao:symbol --> iao:information content entity
  g.add((extensometerSerialNumberIRI, dct.identifier, Literal(extensometerSerialNumber, datatype=XSD.string)))
  g.add((extensometerSerialNumberIRI, obo.IAO_0000219, extensometerIRI)) # Extensometer Serial Number (symbol) iao:denotes Extensometer

  extensometerStandardIRI = URIRef(experimentIRI + "_extensometer_standard")
  g.add((extensometerStandardIRI, a, pmdco.DeviceSpecification)) # is a pmdco:DeviceSpecification as subclass of iao:directive information entity --> iao:information content entity
  g.add((extensometerStandardIRI, dct.identifier, Literal(extensometerStandard, datatype=XSD.string)))
  g.add((extensometerStandardIRI, obo.IAO_0000136, extensometerIRI)) # Extensometer Standard (symbol) iao:is about Extensometer

  # Extensometer Gauge Length
  extensometerGaugeLengthIRI_value_specification = URIRef(experimentIRI + "_extensometerGaugeLength_value_specification")
  g.add((extensometerGaugeLengthIRI_value_specification, a, obo.OBI_0001931)) # is an obi:scalar value specification
  g.add((extensometerGaugeLengthIRI_value_specification, obo.OBI_0001937, Literal(extensometerGaugeLength, datatype=XSD.float)))
  g.add((extensometerGaugeLengthIRI_value_specification, obo.IAO_0000039, unit.MilliM))

  extensometerGaugeLengthIRI = URIRef(experimentIRI + "_extensometerGaugeLength_scalar_measurement_datum")
  g.add((extensometerGaugeLengthIRI, a, obo.IAO_0000032))
  g.add((extensometerGaugeLengthIRI, obo.OBI_0001938, extensometerGaugeLengthIRI_value_specification))

  extensometerGaugeLengthIRI_quality = URIRef(experimentIRI + "_extensometer_gauge_length")
  g.add((extensometerGaugeLengthIRI_quality, a, base.ExtensometerGaugeLength)) # Extensometer Gauge Length used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((extensometerGaugeLengthIRI_quality, obo.IAO_0000417, extensometerGaugeLengthIRI))
  g.add((extensometerGaugeLengthIRI_value_specification, obo.OBI_0001927, extensometerGaugeLengthIRI_quality))
  g.add((extensometerIRI, obo.RO_0000086, extensometerGaugeLengthIRI_quality)) # Extensometer ro:has quality extensometerGaugeLengthIRI_quality

  # Load Cell
  loadCellFunctionIRI = URIRef(experimentIRI + "_load_cell_function")
  g.add((loadCellFunctionIRI, a, obo.OBI_0000453)) # is a obi:measure function
  g.add((loadCellFunctionIRI, obo.BFO_0000054, processIRI)) # measure function (of load cell) is bfo:realized in (tensile testing) process
  g.add((loadCellFunctionIRI, obo.BFO_0000054, tensileTestISOIRI)) # load cell function is bfo:realized in ISO tensile testing process (subprocess)

  loadCellIRI = URIRef(experimentIRI + "_load_cell")
  g.add((loadCellIRI, a, pmdco.LoadCell)) # is a pmdco:LoadCell as subclass of pmdco:ForceMeasuringSystem
  g.add((loadCellIRI, obo.RO_0000085, loadCellFunctionIRI)) # Load cell ro:has function load cell function
  g.add((machineIRI, obo.RO_0002180, loadCellIRI)) # Tensile Testing Machine ro:has component Load cell

  loadCellMaxForce_value_specification = URIRef(experimentIRI + "_loadCellMaxForce_value_specification")
  g.add((loadCellMaxForce_value_specification, a, obo.OBI_0001931)) # is an obi:scalar value specification
  g.add((loadCellMaxForce_value_specification, obo.OBI_0001937, Literal(loadCellMaxForce, datatype=XSD.float)))
  g.add((loadCellMaxForce_value_specification, obo.IAO_0000039, unit.N))

  loadCellMaxForceQualityIRI = URIRef(experimentIRI + "_load_cell_max_force_quality")
  g.add((loadCellMaxForceQualityIRI, a, obo.OBI_0000654)) # is a obi:device setting
  g.add((loadCellMaxForce_value_specification, obo.OBI_0001927, loadCellMaxForceQualityIRI)) # loadCellMaxForce_value_specification obi:specifies value of loadCellMaxForceQuality (device setting)
  g.add((loadCellIRI, obo.RO_0000086, loadCellMaxForceQualityIRI)) # Load cell ro:has quality loadCellMaxForceQuality (device setting)

  # Micrometer Gauge
  micrometerGaugeFunctionIRI = URIRef(experimentIRI + "_micrometer_gauge_function")
  g.add((micrometerGaugeFunctionIRI, a, obo.OBI_0000453)) # is a obi:measure function
  g.add((micrometerGaugeFunctionIRI, obo.BFO_0000054, processIRI)) # measure function (of micrometer gauge) is bfo:realized in (tensile testing) process
  g.add((micrometerGaugeFunctionIRI, obo.BFO_0000054, preMeasuringIRI)) # measure function (of micrometer gauge) is bfo:realized in preMeasuring process (subprocess)
  g.add((micrometerGaugeFunctionIRI, obo.BFO_0000054, postMeasuringIRI)) # measure function (of micrometer gauge) is bfo:realized in postMeasuring process (subprocess)

  micrometerGaugeIRI = URIRef(experimentIRI + "_micrometer_gauge")
  g.add((micrometerGaugeIRI, a, pmdco.MicrometerGauge)) # Micrometer Gauge is a pmdco:MicrometerGauge
  g.add((micrometerGaugeIRI, obo.RO_0000085, micrometerGaugeFunctionIRI)) # Micrometer Gauge ro:has function measure function (of micrometer gauge)
  g.add((micrometerGaugeIRI, obo.RO_0002215, preMeasuringIRI)) # Micrometer Gauge is ro:capable of preMeasuring process (can realize the process)
  g.add((micrometerGaugeIRI, obo.RO_0002215, postMeasuringIRI)) # Micrometer Gauge is ro:capable of postMeasuring process (can realize the process)
  g.add((processIRI, obo.RO_0002608, micrometerGaugeIRI)) # Process (tensile testing) ro:process has causal agent Micrometer Gauge
  g.add((preMeasuringIRI, obo.RO_0002608, micrometerGaugeIRI)) # preMeasuring process ro:process has causal agent Micrometer Gauge
  g.add((postMeasuringIRI, obo.RO_0002608, micrometerGaugeIRI)) # postMeasuring process ro:process has causal agent Micrometer Gauge

  micrometerGaugeManufacturerIRI = URIRef(experimentIRI + "_micrometer_gauge_manufacturer")
  g.add((micrometerGaugeManufacturerIRI, a, pmdco.Manufacturer)) # is a Manufacturer which is a subclass of Organization --> material entity
  g.add((micrometerGaugeManufacturerIRI, dct.identifier, Literal(micrometerGaugeManufacturer, datatype=XSD.string)))
  g.add((micrometerGaugeIRI, obo.OBI_0000304, micrometerGaugeManufacturerIRI)) # Micrometer Gauge obi:is_manufactured_by Manufacturer

  # Caliper
  caliperFunctionIRI = URIRef(experimentIRI + "_caliper_function")
  g.add((caliperFunctionIRI, a, obo.OBI_0000453)) # is a obi:measure function
  g.add((caliperFunctionIRI, obo.BFO_0000054, processIRI)) # measure function (of caliper) is bfo:realized in (tensile testing) process
  g.add((caliperFunctionIRI, obo.BFO_0000054, preMeasuringIRI)) # measure function (of caliper) is bfo:realized in preMeasuring process (subprocess)
  g.add((caliperFunctionIRI, obo.BFO_0000054, postMeasuringIRI)) # measure function (of caliper) is bfo:realized in postMeasuring process (subprocess)

  caliperIRI = URIRef(experimentIRI + "_caliper")
  g.add((caliperIRI, a, pmdco.Caliper)) # Caliper is a pmdco:Caliper
  g.add((caliperIRI, obo.RO_0000085, micrometerGaugeFunctionIRI)) # Caliper ro:has function measure function (of Caliper)
  g.add((caliperIRI, obo.RO_0002215, preMeasuringIRI)) # Caliper is ro:capable of preMeasuring process (can realize the process)
  g.add((caliperIRI, obo.RO_0002215, postMeasuringIRI)) # Caliper is ro:capable of postMeasuring process (can realize the process)
  g.add((processIRI, obo.RO_0002608, caliperIRI)) # Process (tensile testing) ro:process has causal agent Caliper
  g.add((preMeasuringIRI, obo.RO_0002608, caliperIRI)) # preMeasuring process ro:process has causal agent Caliper
  g.add((postMeasuringIRI, obo.RO_0002608, caliperIRI)) # postMeasuring process ro:process has causal agent Caliper

  caliperManufacturerIRI = URIRef(experimentIRI + "_caliper_manufacturer")
  g.add((caliperManufacturerIRI, a, pmdco.Manufacturer)) # is a Manufacturer which is a subclass of Organization --> material entity
  g.add((caliperManufacturerIRI, dct.identifier, Literal(caliperManufacturer, datatype=XSD.string)))
  g.add((caliperIRI, obo.OBI_0000304, caliperManufacturerIRI)) # Caliper obi:is_manufactured_by Manufacturer

  caliperSerialNumberIRI = URIRef(experimentIRI + "_caliper_serial_number")
  g.add((caliperSerialNumberIRI, a, obo.IAO_0000131)) # is an iao:serial number as subclass of iao:symbol --> iao:information content entity
  g.add((caliperSerialNumberIRI, dct.identifier, Literal(caliperSerialNumber, datatype=XSD.string)))
  g.add((caliperSerialNumberIRI, obo.IAO_0000219, caliperIRI)) # Caliper Serial Number (symbol) iao:denotes Caliper

  # Analysis Software Application
  softwareApplicationIRI = URIRef(experimentIRI + "_software_application")
  g.add((softwareApplicationIRI, a, obo.IAO_0000594)) # is an iao:software application as a subclass of software --> plan specification
  g.add((processIRI, obo.STATO_0000102, softwareApplicationIRI)) # Tensile Test process stato:executes Software Application
  g.add((analysisIRI, obo.STATO_0000102, softwareApplicationIRI)) # Analysis process (subprocess of tensile test) stato:executes Software Application

  softwareApplicationNameIRI = URIRef(experimentIRI + "_software_application_identifier")
  g.add((softwareApplicationNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((softwareApplicationNameIRI, dct.identifier, Literal("BAM Tensile Test Analysis App", datatype=XSD.string)))
  g.add((softwareApplicationNameIRI, obo.IAO_0000219, softwareApplicationIRI)) # Software Application Name (identifier) iao:denotes Software Application

# Graph Serialization

In [13]:
# Graph g is serialized in two different data formats / notations, Turtle (ttl) and RDF
g.serialize("RDF-TTO-data_S355.ttl", format="ttl")
g.serialize("RDF-TTO-data_S355.rdf", format='application/rdf+xml', encoding="utf-8-sig")

# Instead of serializing the data to be stored in the working directory, 
# the files could alternatively be stored in another repository or directly
# uploaded to a triple store.

<Graph identifier=Ncbe360db06c34ae5afdfe8ababcef87e (<class 'rdflib.graph.Graph'>)>

# Information concerning Namespaces and Concepts

As some ontologies concepts of which are used within this script do not use IRIs that can directly be interpreted by humans (humans will have to read the labels provided), the corresponding labels are given, here, for the most important concepts frequently used in this script. 

### Basic Formal Ontology (BFO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:BFO_0000063](http://purl.obolibrary.org/obo/BFO_0000063)       | precedes         | Object Property |
| [obo:BFO_0000055](http://purl.obolibrary.org/obo/BFO_0000055)       | realizes         | Object Property |
| [obo:BFO_0000054](http://purl.obolibrary.org/obo/BFO_0000054)       | realized in         | Object Property |
| [obo:BFO_0000051](http://purl.obolibrary.org/obo/BFO_0000051)       | has part         | Object Property |


### Information Artifact Ontology (IAO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:IAO_0020000](http://purl.obolibrary.org/obo/IAO_0020000)       | identifier         | Class |
| [obo:IAO_0000032](http://purl.obolibrary.org/obo/IAO_0000032)       | scalar measurement datum         | Class |
| [obo:IAO_0000003](http://purl.obolibrary.org/obo/IAO_0000003)       | measurement unit label         | Class |
| [obo:IAO_0000027](http://purl.obolibrary.org/obo/IAO_0000027)       | data item         | Class |
| [obo:IAO_0000131](http://purl.obolibrary.org/obo/IAO_0000131)       | serial number         | Class |
| [obo:IAO_0000594](http://purl.obolibrary.org/obo/IAO_0000594)       | software application         | Class |
| [obo:IAO_0000100](http://purl.obolibrary.org/obo/IAO_0000100)       | data set         | Class |
| [obo:IAO_0000416](http://purl.obolibrary.org/obo/IAO_0000416)       | time measurement datum         | Class |
| [obo:IAO_0000300](http://purl.obolibrary.org/obo/IAO_0000300)       | textual entity         | Class |
| [obo:IAO_0000422](http://purl.obolibrary.org/obo/IAO_0000422)       | postal address         | Class |
| [obo:IAO_0000578](http://purl.obolibrary.org/obo/IAO_0000578)       | centrally registered identifier         | Class |
| [obo:IAO_0000039](http://purl.obolibrary.org/obo/IAO_0000039)       | has measurement unit label         | Object Property |
| [obo:IAO_0000417](http://purl.obolibrary.org/obo/IAO_0000417)       | is quality measured as         | Object Property |
| [obo:IAO_0000136](http://purl.obolibrary.org/obo/IAO_0000136)       | is about         | Object Property |
| [obo:IAO_0000219](http://purl.obolibrary.org/obo/IAO_0000219)       | denotes         | Object Property |
| [obo:IAO_0000115](http://purl.obolibrary.org/obo/IAO_0000115)       | has textual content         | Object Property |


### Ontology for Biomedical Investigations (OBI)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:OBI_0000011](http://purl.obolibrary.org/obo/OBI_0000011)       | planned process         | Class |
| [obo:OBI_0000070](http://purl.obolibrary.org/obo/OBI_0000070)       | assay         | Class |
| [obo:OBI_0001931](http://purl.obolibrary.org/obo/OBI_0001931)       | scalar value specification         | Class |
| [obo:OBI_0000260](http://purl.obolibrary.org/obo/OBI_0000260)       | plan         | Class |
| [obo:OBI_0000104](http://purl.obolibrary.org/obo/IAO_0000104)       | plan specification        | Class |
| [obo:OBI_0000835](http://purl.obolibrary.org/obo/OBI_0000835)       | manufacturer        | Class |
| [obo:OBI_0000571](http://purl.obolibrary.org/obo/OBI_0000571)       | manufacturer role       | Class |
| [obo:OBI_0000453](http://purl.obolibrary.org/obo/OBI_0000453)       | measure function       | Class |
| [obo:OBI_0000654](http://purl.obolibrary.org/obo/OBI_0000654)       | device setting       | Class |
| [obo:OBI_0001937](http://purl.obolibrary.org/obo/OBI_0001937)       | has specified numeric value         | Datatype Property |
| [obo:OBI_0002815](http://purl.obolibrary.org/obo/OBI_0002815)       | has representation         | Datatype Property |
| [obo:OBI_0000293](http://purl.obolibrary.org/obo/OBI_0000293)       | has specified input         | Object Property |
| [obo:OBI_0000299](http://purl.obolibrary.org/obo/OBI_0000299)       | has specified output         | Object Property |
| [obo:OBI_0001927](http://purl.obolibrary.org/obo/OBI_0001927)       | specifies value of         | Object Property |
| [obo:OBI_0001938](http://purl.obolibrary.org/obo/OBI_0001938)       | has value specification         | Object Property |
| [obo:OBI_0000304](http://purl.obolibrary.org/obo/OBI_0000304)       | is_manufactured_by         | Object Property |


 ### Relation Ontology (RO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:RO_0000086](http://purl.obolibrary.org/obo/RO_0000086)       | has quality         | Object Property |
| [obo:RO_0002090](http://purl.obolibrary.org/obo/RO_0002090)       | immediately precedes         | Object Property |
| [obo:RO_0002012](http://purl.obolibrary.org/obo/RO_0002012)       | occurrent part of         | Object Property |
| [obo:RO_0002224](http://purl.obolibrary.org/obo/RO_0002224)       | starts with         | Object Property |
| [obo:RO_0001000](http://purl.obolibrary.org/obo/RO_0001000)       | derives from        | Object Property |
| [obo:RO_0000059](http://purl.obolibrary.org/obo/RO_0000059)       | concretizes        | Object Property |
| [obo:RO_0002180](http://purl.obolibrary.org/obo/RO_0002180)       | has component        | Object Property |
| [obo:RO_0000085](http://purl.obolibrary.org/obo/RO_0000085)       | has function        | Object Property |
| [obo:RO_0002215](http://purl.obolibrary.org/obo/RO_0002215)       | capable of        | Object Property |
| [obo:RO_0002608](http://purl.obolibrary.org/obo/RO_0002608)       | process has causal agent       | Object Property |
| [obo:RO_0000057](http://purl.obolibrary.org/obo/RO_0000057)       | has participant       | Object Property |
| [obo:RO_0000087](http://purl.obolibrary.org/obo/RO_0000087)       | has role       | Object Property |
| [obo:RO_0002566](http://purl.obolibrary.org/obo/RO_0002566)       | causally influences       | Object Property |


### The Statistical Methods Ontology (STATO)
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:STATO_0000102](http://purl.obolibrary.org/obo/STATO_0000102)       | executes         | Object Property |


### Time Ontology in OWL
*(Namespace: [time:](http://www.w3.org/2006/time#)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [time:Duration](http://www.w3.org/2006/time#Duration)       | Duration         | Class |
| [time:Instant](http://www.w3.org/2006/time#Instant)       | Instant         | Class |
| [time:hasTime](http://www.w3.org/2006/time#hasTime)       | has time         | Object Property |
| [time:inXSDDate](http://www.w3.org/2006/time#inXSDDate)       | in XSD Date         | Datatype Property |
| [time:inXSDDateTime](http://www.w3.org/2006/time#inXSDDateTime)       | in XSD Date Time         | Datatype Property |